 # Analyze MFC used cars

## Problem Statement


## Objective


## Business Benefits



## Dataset Descriptions



# Data Loading and Analysis

## Loading the libraries and initialization of basic variables

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime 

In [ ]:
# data file paths
JTD_DATA_FILE = "../data/JTD.csv"
CUST_DATA_FILE = "../data/Customer_Data.xlsx"
PLANT_MASTER_FILE = "../data/Plant Master.xlsx"

# log level
LOG_LEVEL = 'INFO'

# Year for Age calculation
BASELINE_YEAR = 2018

In [ ]:
# Date today
#DATE_FOR_CALC = datetime(2018,8,22,0,0,0)
#DATE_FOR_CALC

## Data Loading

In [ ]:
# load the data files for JTD, Customer and Plant data
jtd_df_orig = pd.read_csv(JTD_DATA_FILE) 
cust_df_orig = pd.read_excel(CUST_DATA_FILE)
plant_df_orig = pd.read_excel(PLANT_MASTER_FILE)

### Rename the columns to standardize them

In [ ]:
# renaming columns to standardize them
# convert them to lower case and remove space, use _ for separating entity names
# rename JTD columns
jtd_df = jtd_df_orig.rename(columns={"DBM Order":"dbm_order",\
                                "Order Item":"order_item",\
                                "Material":"material",\
                                "Labor Value Number":"labor_value_number",\
                                "Description":"description",\
                                "Item Category":"item_category",\
                                "Order Quantity":"order_quantity",\
                                "Target quantity UoM":"target_quantity_uom",\
                                "Net value":"net_value"})

# rename Customer columns
cust_df = cust_df_orig.rename(columns={"Business Partner":"business_partner",\
                                       "Customer No.":"customer_no",\
                                       "Partner Type":"partner_type",\
                                       "Data Origin":"data_origin",\
                                       "Title":"title",\
                                       "Marital Status":"marital_status",\
                                       "Occupation":"occupation",\
                                       "Date of Birth":"date_birth",\
                                       "Death date":"date_demise"})

# rename Plant Master columns
plant_df = plant_df_orig.rename(columns={"Plant":"plant",\
                                         "Name 1":"name_1",\
                                         "Valuation Area":"valuation_area",\
                                         "Customer no. - plant":"customer_no_plant",\
                                         "Vendor number plant":"vendor_no_plant",\
                                         "Factory calendar":"factory_calendar",\
                                         "Name 2":"name_2",\
                                         "House number and street":"house_no_street",\
                                         "PO Box":"po_box",\
                                         "Postal Code":"postal_code",\
                                         "City":"city",\
                                         "Sales organization":"sales_org",\
                                         "State":"state"})

### Basic Analysis based on number of records in each dataset

In [ ]:
# check the number of records from all the three datasets
print('Number of records:')
print('JTD Records (Service records from JTD): {:,}'.format(jtd_df.shape[0]))
print('Customer Records (Customer records from Customer): {:,}'.format(cust_df.shape[0]))
print('Plant Records (Plant records from Plant file): {:,}'.format(plant_df.shape[0]))

In [ ]:
def printBasicInfo(data, briefDesc):
    if LOG_LEVEL == 'INFO':
        print('Dataframe Brief Description for:', briefDesc)
        print('======='*12)
        print('\nDataframe Field Types')
        print(data.info())
        print('\nDataframe Field Null and NonNull fields')
        print(data.isnull().sum())
        print('\nDataframe: ',briefDesc)
        display(data.head(5))

In [ ]:
printBasicInfo(jtd_df, briefDesc='JTD Dataset information')

### Customer Dataset - Add columns to indicate is_alive and age of the customer

In [ ]:
printBasicInfo(cust_df, briefDesc='Customer Dataset information')

In [ ]:
cust_df['date_birth'] = pd.to_datetime(cust_df['date_birth'], errors = 'coerce')

In [ ]:
cust_df['date_demise'].isnull().value_counts()

In [ ]:
# Calculates the age of the customer in years
def calculateAgeInYears(dtBirth, dtDemise):
    # default age if date of birth and demise are not available
    ageVal = 199
    # if both date of birth and date of demise is available
    # then use these values to calculate age
    if (not pd.isnull(dtBirth)) & (not pd.isnull(dtDemise)):
         ageVal = dtDemise.year - dtBirth.year
    
    # if both date of demise is not available, assumed that person is alive
    # then use current baseline year to calculate age in years 
    # based on the birth date
    elif (not pd.isnull(dtBirth)) & pd.isnull(dtDemise):
         ageVal = BASELINE_YEAR - dtBirth.year
    return ageVal


# Group all the logic required for customer dataset feature
# preprocessing in this function block
def preprocessCustFeatures(customerData):
    # creates a new feature called is_alive to check if person is alive
    customerData['is_alive'] = customerData['date_demise'].isnull()
    # creates a new feature called age based on current baseline year
    customerData['age'] = customerData.apply(lambda x: calculateAgeInYears(\
                                            x.date_birth,\
                                            x.date_demise), axis=1)
    return customerData

In [ ]:
cust_df = preprocessCustFeatures(customerData=cust_df)
cust_df.info()

In [ ]:
printBasicInfo(plant_df, briefDesc='Plant Dataset information')

In [ ]:
# Number of customer numbers not in customer records file
print('Number of Customers from Plant Master who are not present in Customer Master:')
display(plant_df['customer_no_plant']\
        .isin(cust_df['customer_no'])\
        .value_counts())

# List of such unique Number of customer numbers not in customer records file

orphPlanCustList = pd.unique(plant_df[plant_df['customer_no_plant']\
                   .isin(cust_df['customer_no']) == False]\
                    .customer_no_plant)
print('\nTotal number of customer records in plant master \
which are not present in customer master:',len(orphPlanCustList))
print('\nSample orphan Customer Records in Plant: ',orphPlanCustList[1:5])

<font color='blue'><b>Observation</b>:As we can see from above that all the customer numbers mentioned in the Plant Master does not have corresponding records in Customer Master.</font>